# Sparkxperiment
This notebook will be used for experiment purpose in order to work on my assignment

## Import Libs

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
# Initate spark session
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("sparkxperiment") \
    .getOrCreate()

22/12/10 23:24:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

## Load Data

In [4]:
# See current dir
!pwd

/home/jovyan/work


In [5]:
df_fhv_trip = (
    spark.read
    .format("parquet")
    .option("header", True)
    .load("/home/jovyan/work/data/fhv_tripdata_2022-02.parquet")
)

In [8]:
df_fhv_trip.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00014|2022-02-01 00:15:00|2022-02-01 00:31:00|        null|        null|   null|                B00014|
|              B00021|2022-02-01 00:08:40|2022-02-01 00:42:54|        95.0|       173.0|   null|       B00021         |
|              B00021|2022-02-01 00:45:26|2022-02-01 01:04:37|       192.0|        70.0|   null|       B00021         |
|              B00021|2022-02-01 00:50:39|2022-02-01 01:29:06|       173.0|       147.0|   null|       B00021         |
|              B00021|2022-02-01 00:22:17|2022-02-01 00:27:29|       129.0|       129.0|   null|       B00021         |
|              B00021|2022-02-01 00:30:3

In [6]:
df_fhv_trip.createOrReplaceTempView('fhv_trip')

## 1. How many taxi trips were there on February 15?

### With Spark SQL

In [24]:
df_result1_sql = spark.sql(
"""
SELECT count(*)
FROM fhv_trip
WHERE pickup_datetime LIKE "%2022-02-15%"
""")

In [25]:
df_result1_sql.show()

+--------+
|count(1)|
+--------+
|   51310|
+--------+



### With Spark

In [26]:
df_result1_spark = (df_fhv_trip.select('dispatching_base_num') \
    .filter(df_fhv_trip.pickup_datetime.contains("2022-02-15")) \
    .count())

In [27]:
df_result1_spark

51310

## 2. Find the longest trip for each day ?

## 3. Find Top 5 Most frequent `dispatching_base_num` ?

### With Spark SQL

In [20]:
df_result3_sql = spark.sql(
"""
WITH five_most_freq AS(
SELECT dispatching_base_num, COUNT(dispatching_base_num) AS freq
FROM fhv_trip
GROUP BY dispatching_base_num
ORDER BY freq DESC
LIMIT 5
)

SELECT *
FROM five_most_freq
""")

In [21]:
df_result3_sql.show()

+--------------------+-----+
|dispatching_base_num| freq|
+--------------------+-----+
|              B03284|84472|
|              B00856|47567|
|              B02799|33214|
|              B03016|31220|
|              B01145|30468|
+--------------------+-----+



## 4. Find Top 5 Most common location pairs (PUlocationID and DOlocationID)?

In [ ]:
def crazy_stuff(pu_loc, do_loc):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'